In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import h5py
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Convolution1D, Dense, Dropout, MaxPooling1D, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from keras import callbacks
from keras.callbacks import CSVLogger
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [0]:
np.random.seed(1337)

In [0]:
x = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/train_data.csv')
y = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/train_label.csv')

In [0]:
X = x.iloc[:, 0:42]
Y = y.iloc[:]
lb = LabelEncoder()
Y = lb.fit_transform(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)
scaler = Normalizer().fit(X_train)
X_train = scaler.transform(X_train)
scaler = Normalizer().fit(X_test)
X_test = scaler.transform(X_test)

In [0]:
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [0]:
lstm_output_size = 70

In [0]:
cnn = Sequential()
cnn.add(Convolution1D(64, 3, activation="relu", input_shape=(41, 1)))
cnn.add(Convolution1D(64, 3, activation="relu"))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Convolution1D(128, 3, activation="relu"))
cnn.add(Convolution1D(128, 3, activation="relu"))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(LSTM(lstm_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(5, activation="softmax"))

In [0]:
cnn.compile(loss="sparse_categorical_crossentropy", optimizer="SGD", 
            metrics=['accuracy'])
print(cnn.summary())
checkpointer = callbacks.ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/HDF5 Models/checkpoint-{epoch:02d}.hdf5", 
                                         verbose=1, save_best_only=True, 
                                         monitor='val_acc',mode='max')
csv_logger = CSVLogger('/content/drive/My Drive/Colab Notebooks/cnntrainanalysis3.csv',separator=',', append=False)
cnn.fit(X_train, y_train, epochs=1000, batch_size=10, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("/content/drive/My Drive/Colab Notebooks/cnn_model.hdf5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 39, 64)            256       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 37, 64)            12352     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 18, 64)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 16, 128)           24704     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 14, 128)           49280     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 7, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 70)                55720     
__________

ValueError: ignored